In [5]:
from typing import List, Tuple
import subprocess
import itertools

# alias de types
Variable = int
Literal = int
Clause = List[Literal]
Model = List[Literal]
Clause_Base = List[Clause]
Grid = List[List[int]]

SIZE = 4

In [6]:
Quatre_Queen: Grid = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
]

In [7]:
def at_least_one(vars: List[int]) -> List[int]:
    return [i for i in vars]

In [8]:
def unique(vars: List[int]) -> List[List[int]]:
    base = [list((-l1, -l2)) for l1, l2 in itertools.combinations(vars, 2)]
    base.append(at_least_one(vars))
    return base

In [9]:
def AtmostOne(vars: List[int]):
    base = unique(vars)
    base.pop()
    return base

In [10]:
AtmostOne([1, 2, 3, 4])

[[-1, -2], [-1, -3], [-1, -4], [-2, -3], [-2, -4], [-3, -4]]

In [11]:
def row_col_to_var(row,col):
    #global first_var
    return row*SIZE+col+1

In [14]:
def var_to_row_col(var):
    i = (var-1) // SIZE
    j = (var-1) % SIZE
    return (i,j)


In [18]:
row_col_to_var(1,2)

7

In [19]:
var_to_row_col(7)


(1, 2)

one single queen must be present at each row

In [10]:
def create_line_constraints() -> List[List[int]]:
    base = []
    id_j = 1
    for i in range(0,SIZE):
            base += unique([j for j in range(id_j,SIZE+id_j)])
            id_j += SIZE
    return base

In [11]:
create_line_constraints()

[[-1, -2],
 [-1, -3],
 [-1, -4],
 [-2, -3],
 [-2, -4],
 [-3, -4],
 [1, 2, 3, 4],
 [-5, -6],
 [-5, -7],
 [-5, -8],
 [-6, -7],
 [-6, -8],
 [-7, -8],
 [5, 6, 7, 8],
 [-9, -10],
 [-9, -11],
 [-9, -12],
 [-10, -11],
 [-10, -12],
 [-11, -12],
 [9, 10, 11, 12],
 [-13, -14],
 [-13, -15],
 [-13, -16],
 [-14, -15],
 [-14, -16],
 [-15, -16],
 [13, 14, 15, 16]]

In [12]:
def create_columns_constraints() -> List[List[int]]:
    base = []
    id_i = 1
    for j in range(SIZE):
            base += unique([i for i in range(j+1,SIZE**2-(SIZE-1)+j+1, SIZE)])
            #id_i += size_grid
    return base

In [13]:
create_columns_constraints()

[[-1, -5],
 [-1, -9],
 [-1, -13],
 [-5, -9],
 [-5, -13],
 [-9, -13],
 [1, 5, 9, 13],
 [-2, -6],
 [-2, -10],
 [-2, -14],
 [-6, -10],
 [-6, -14],
 [-10, -14],
 [2, 6, 10, 14],
 [-3, -7],
 [-3, -11],
 [-3, -15],
 [-7, -11],
 [-7, -15],
 [-11, -15],
 [3, 7, 11, 15],
 [-4, -8],
 [-4, -12],
 [-4, -16],
 [-8, -12],
 [-8, -16],
 [-12, -16],
 [4, 8, 12, 16]]

In [14]:
"""
def gen_diagonal(start_row,start_col,increment,limit):
    col=start_col
    tmp=[]
    for row in range(start_row,SIZE):
        tmp.append(row_col_to_var(row,col))
        col=col+increment
        if col==limit:
            break
            #ignore diagonal sconsisting of 1 cell:
    if len(tmp)>1:
            # we can't use POPCNT1()here,sincesomediagonalsareemptyinvalidsolutions.
        unique(tmp).pop()
        return tmp
"""

"\ndef gen_diagonal(start_row,start_col,increment,limit):\n    col=start_col\n    tmp=[]\n    for row in range(start_row,SIZE):\n        tmp.append(row_col_to_var(row,col))\n        col=col+increment\n        if col==limit:\n            break\n            #ignore diagonal sconsisting of 1 cell:\n    if len(tmp)>1:\n            # we can't use POPCNT1()here,sincesomediagonalsareemptyinvalidsolutions.\n        unique(tmp).pop()\n        return tmp\n"

In [15]:
#gen_diagonal(0,0, 1,SIZE)

In [16]:
"""
def create_diago_constraints() -> List[List[int]]:
    for row in range(SIZE):
        for col in range(SIZE):
            base1 = gen_diagonal(row,col, 1,SIZE) # from L to R
            base2 = gen_diagonal(row,col, -1, -1) # from R to L

    return base1 + base2
"""

'\ndef create_diago_constraints() -> List[List[int]]:\n    for row in range(SIZE):\n        for col in range(SIZE):\n            base1 = gen_diagonal(row,col, 1,SIZE) # from L to R\n            base2 = gen_diagonal(row,col, -1, -1) # from R to L\n\n    return base1 + base2\n'

In [57]:
def create_diago_constraints() -> List[List[int]]:
  base = []

  # de la diagonale gauche(haut)-droite(bas) du centre vers le coin supérieur de la partie droite
  base1 = []
  for i in range(SIZE-1):
    for j in range(SIZE-i):
      base1.append(row_col_to_var(j,i+j))
    #print(base1)
    # pour avoir simplement une liste de liste et non une liste de liste de liste
    base1 = AtmostOne(base1)
    for i in base1:
        base.append(i)
    base1 = []

# après la diagonale gauche-droite du centre vers le coin inférieur de la partie gauche
  base2 = []
  for i in range(1,SIZE-1):
    for j in range(SIZE-i):
      base2.append(row_col_to_var(j+i,j))
    base2 = AtmostOne(base2)
    for i in base2:
        base.append(i)
    base2 = []

  # de la diagonale droite(haut)-gauche(bas) du centre vers le coin supérieur de la partie gauche
  base3 = []
  for i in range(SIZE-1):
    for j in range(SIZE-i):
      base3.append(row_col_to_var(j,SIZE-1-(i+j)))
    base3 = AtmostOne(base3)
    for i in base3:
        base.append(i)
    base3 = []

# après la diagonale droite(haut)-gauche(bas) du centre vers le coin inférieur de la partie droite
  base4 = []
  for i in range(1,SIZE-1):
    for j in range(SIZE-i):
      base4.append(row_col_to_var(j+i,SIZE-1-j))
    base4 = AtmostOne(base4)
    for i in base4:
        base.append(i)
    base4 = []


  #base = base1 + base2 + base3 + base4
  return base


In [58]:
create_diago_constraints()


[[-1, -6],
 [-1, -11],
 [-1, -16],
 [-6, -11],
 [-6, -16],
 [-11, -16],
 [-2, -7],
 [-2, -12],
 [-7, -12],
 [-3, -8],
 [-5, -10],
 [-5, -15],
 [-10, -15],
 [-9, -14],
 [-4, -7],
 [-4, -10],
 [-4, -13],
 [-7, -10],
 [-7, -13],
 [-10, -13],
 [-3, -6],
 [-3, -9],
 [-6, -9],
 [-2, -5],
 [-8, -11],
 [-8, -14],
 [-11, -14],
 [-12, -15]]

In [111]:
def gen_diagonal(start_row,start_col,increment,limit):
    col=start_col
    tmp=[]
    for row in range(start_row,SIZE):
        tmp.append(row_col_to_var(row,col))
        col=col+increment
        if col==limit:
            break
            #ignore diagonal sconsisting of 1 cell:
    if len(tmp)>1:
            # we can't use POPCNT1()here,since some diagonals are empty invalid solutions.

        tmp = AtmostOne(tmp)
        return tmp

In [112]:
def create_diago_constraints() -> List[List[int]]:
    base1 = []
    base2 = []
    for row in range(SIZE):
        for col in range(SIZE):
            base1.append(gen_diagonal(row,col, 1,SIZE)) # from L to R
            base2.append(gen_diagonal(row,col, -1, -1))# from R to L
    #print(base1)
    #print(base2)
    base = []
    for i in base1 + base2:
        if i != None:
            base.append(i)
    # pour supprimer la liste de liste de liste et pour avoir une liste de liste
    base_finale = []
    for i in base:
        for j in i:
            base_finale.append(j)
    # enlever les doublons !
    base_finale2  = []
    for i in base_finale:
        if i not in base_finale2: base_finale2.append(i)

    return base_finale2

In [113]:
"""
def create_diago_constraints() -> List[List[int]]:
    base = []
    for j in range(SIZE):
        base += AtmostOne([j + i for i in range(SIZE)])
        base += AtmostOne([j - i for i in range(SIZE)])
    return base
"""

'\ndef create_diago_constraints() -> List[List[int]]:\n    base = []\n    for j in range(SIZE):\n        base += AtmostOne([j + i for i in range(SIZE)])\n        base += AtmostOne([j - i for i in range(SIZE)])\n    return base\n'

In [114]:
create_diago_constraints()

[[-1, -6],
 [-1, -11],
 [-1, -16],
 [-6, -11],
 [-6, -16],
 [-11, -16],
 [-2, -7],
 [-2, -12],
 [-7, -12],
 [-3, -8],
 [-5, -10],
 [-5, -15],
 [-10, -15],
 [-9, -14],
 [-2, -5],
 [-3, -6],
 [-3, -9],
 [-6, -9],
 [-4, -7],
 [-4, -10],
 [-4, -13],
 [-7, -10],
 [-7, -13],
 [-10, -13],
 [-8, -11],
 [-8, -14],
 [-11, -14],
 [-12, -15]]

In [115]:
def generate_problem(grid: List[List[int]]) -> List[List[int]]:
    base = []
    base += create_line_constraints()
    base += create_columns_constraints()
    base += create_diago_constraints()

    return base

In [116]:
"""
def clauses_to_dimacs(clauses: List[List[int]], nb_vars: int) -> str:
    dimacs_format = "p cnf 16 148\n"
    for listes in clauses:
        for j in listes:
            dimacs_format+=str(j)+" "
        dimacs_format+="0\n"
    return  dimacs_format
"""

'\ndef clauses_to_dimacs(clauses: List[List[int]], nb_vars: int) -> str:\n    dimacs_format = "p cnf 16 148\n"\n    for listes in clauses:\n        for j in listes:\n            dimacs_format+=str(j)+" "\n        dimacs_format+="0\n"\n    return  dimacs_format\n'

In [117]:
def clauses_to_dimacs(grid: List[List[int]], nb_vars : int) -> str:
    dimacs_str = f"c 4_queen\n"
    dimacs_str += f"p cnf {nb_vars} {len(grid)}\n"
    for clause in grid:
        for litt in clause:
            dimacs_str += f"{litt} "

        dimacs_str += "0\n"
    return dimacs_str

In [118]:
def write_dimacs_file(dimacs: str, filename: str):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)


def exec_gophersat(filename: str, cmd: str = "./gophersat-1.1.6", encoding: str = "utf8") -> Tuple[bool, List[int]]:
    result = subprocess.run(
        [cmd, filename], capture_output=True, check=True, encoding=encoding
    )
    string = str(result.stdout)
    lines = string.splitlines()

    if lines[1] != "s SATISFIABLE":
        return False, []

    model = lines[2][2:].split(" ")

    return True, [int(x) for x in model]


In [119]:
def resolution_queen(grid:List[List]) -> List[List]:
    queen = generate_problem(grid) # IMPORTANT ME GENÉRER UN TABLEAU NON VOULU. CELA NE LE FAIT PAS QUAND JE NE MET PAS SUDOKU=
    mes_clauses = clauses_to_dimacs(queen,16)
    write_dimacs_file(mes_clauses,"my_4_queen_clean.cnf")
    mon_execution = exec_gophersat("my_4_queen_clean.cnf")
    #resultat = model_to_grid(mon_execution[1])
    return mon_execution


In [120]:
resolution_queen(Quatre_Queen)

(True, [-1, -2, 3, -4, 5, -6, -7, -8, -9, -10, -11, 12, -13, 14, -15, -16, 0])

In [34]:
create_diago_constraints()

[[1, 6, 11, 16],
 [2, 7, 12],
 [3, 8],
 [5, 10, 15],
 [6, 11, 16],
 [7, 12],
 [9, 14],
 [10, 15],
 [11, 16],
 [2, 5],
 [3, 6, 9],
 [4, 7, 10, 13],
 [6, 9],
 [7, 10, 13],
 [8, 11, 14],
 [10, 13],
 [11, 14],
 [12, 15]]